**Import libraries**

In [ ]:
!pip install pandas-profiling==2.7.1
!pip install scikit-plot

import numpy as np
import pandas as pd
import seaborn as sns
import pandas_profiling
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
# Classifiers
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
# Folds
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
# Training methods
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
# Metrics
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
# Plots
import matplotlib.pyplot as plt
import scikitplot as skplt
# Balance
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek
from imblearn.ensemble import BalancedBaggingClassifier
# Download dataset
#from google.colab import files

from collections import Counter

**Clean and Prepare Dataset**

In [ ]:
# Read dataset as Dataframe from Github
train_dataset = pd.read_csv("https://raw.githubusercontent.com/hernanChain/csvFiles/master/CreditDatasets/train_dataset.csv")
test_dataset = pd.read_csv("https://raw.githubusercontent.com/hernanChain/csvFiles/master/CreditDatasets/test_dataset.csv")

Show the head of Dataset, first 5 rows

In [ ]:
train_dataset.head()

In [ ]:
test_dataset.head()

Show type of each variable

In [ ]:
print("Numero Filas y Columnas en dataset de entrenamiento = ",train_dataset.shape)
print("Valores nulos en dataSet de entrenamiento: \n")
print(train_dataset.isnull().sum())
print("\nTipos de valores en el dataset \n")
print(train_dataset.dtypes)

In [ ]:
print("Numero Filas y Columnas en dataset de prueba = ",test_dataset.shape)
print("Valores nulos en dataSet de prueba: \n")
print(test_dataset.isnull().sum())
print("\nTipos de valores en el dataset \n")
print(test_dataset.dtypes)

Show extra information

In [ ]:
# How much companies in entire dataset
len(train_dataset.company.unique().tolist())

In [ ]:
# How much rows with each class
print(pd.value_counts(train_dataset['defaulted_loan']))

Convert some variable to numeric values

In [ ]:
def toMapBoolean(chain):
  if chain == True:
    return 1
  elif chain == False:
    return 0
    
def toMapClass(chain):
  if chain == 1:
    return "1"
  elif chain == 0:
    return "0"

def toMapProvider(chain):
  if chain.find("JCB")>= 0:
    return 1
  elif chain.find("VISA") >= 0:
    return 2
  elif chain.find("Maestro") >= 0:
    return 3
  elif chain.find("Diners") >= 0:
    return 4
  elif chain.find("American") >= 0:
    return 5
  elif chain.find("Voyager") >= 0:
    return 6
  elif chain.find("Discover") >= 0:
    return 7
  elif chain.find("Mastercard") >= 0:
    return 8
  else:
    return 9

def toMapDateApplicationYear(chain):
  return int(chain[0:4])

def toMapDateApplicationMonth(chain):
  return int(chain[5:7])

def toMapEmail(chain):
  if chain == "gmail.com":
    return 1
  elif chain == "hotmail.com":
    return 2
  elif chain == "yahoo.com":
    return 3
  else:
    return 4

def toMapExpireMonth(chain):
  return int(chain[0:2])

def splitCurrentsJobs(chain):
  currentJobs = chain.split(sep=',')
  return currentJobs

def getFirstJob(chain):
  currentJobs = splitCurrentsJobs(chain)
  return currentJobs[0].lower()

def getSecondJob(chain):
  currentJobs = splitCurrentsJobs(chain)
  if len(currentJobs) > 1:
    return currentJobs[1][1:len(currentJobs[1])].lower()
  else:
    return 'nothing'

In [ ]:
# True -> 1
# False -> 0
train_dataset['defaulted_loan'] = train_dataset['defaulted_loan'].apply(toMapBoolean)
train_dataset['has_criminal_records'] = train_dataset['has_criminal_records'].apply(toMapBoolean)
train_dataset['is_married'] = train_dataset['is_married'].apply(toMapBoolean)
train_dataset['smoker'] = train_dataset['smoker'].apply(toMapBoolean)

test_dataset['has_criminal_records'] = test_dataset['has_criminal_records'].apply(toMapBoolean)
test_dataset['is_married'] = test_dataset['is_married'].apply(toMapBoolean)
test_dataset['smoker'] = test_dataset['smoker'].apply(toMapBoolean)

# 1 ->  JCB x digit
# 2 ->  VISA x digit
# 3 ->  Maestro
# 4 ->  Diners Club / Carte Blanche
# 5 ->  American Express
# 6 ->  Voyager
# 7 ->  Discover
# 8 ->  Mastercard
# 9 ->  Otro

train_dataset['credit_card_provider'] = train_dataset['credit_card_provider'].apply(toMapProvider)

test_dataset['credit_card_provider'] = test_dataset['credit_card_provider'].apply(toMapProvider)

# Insert in column 7 only application year

train_dataset.insert(7,'application_year', train_dataset['application_date'].apply(toMapDateApplicationYear),True)

test_dataset.insert(7,'application_year', test_dataset['application_date'].apply(toMapDateApplicationYear),True)

# Insert in column 8 only application month

train_dataset.insert(8,'application_month',train_dataset['application_date'].apply(toMapDateApplicationMonth),True)

test_dataset.insert(8,'application_month',test_dataset['application_date'].apply(toMapDateApplicationMonth),True)

# 1 ->  gmail.com
# 2 ->  hotmail.com
# 3 ->  yahoo.com
# 4 ->  Otro

train_dataset['email_domain'] = train_dataset.email_domain.apply(toMapEmail)

test_dataset['email_domain'] = test_dataset.email_domain.apply(toMapEmail)

# Insert in column 18 only expire month

train_dataset.insert(18,'expire_month', train_dataset['credit_card_expire'].apply(toMapExpireMonth),True)

test_dataset.insert(18,'expire_month', test_dataset['credit_card_expire'].apply(toMapExpireMonth),True)

# Create two new columns with jobs

train_dataset.insert(15,'first_job', train_dataset.current_job.apply(getFirstJob),True)
train_dataset.insert(16,'second_job', train_dataset.current_job.apply(getSecondJob),True)

test_dataset.insert(15,'first_job', test_dataset.current_job.apply(getFirstJob),True)
test_dataset.insert(16,'second_job', test_dataset.current_job.apply(getSecondJob),True)

In [ ]:
jobs1 = train_dataset['first_job'].unique().tolist()
jobs2 = train_dataset['second_job'].unique().tolist()
jobs = jobs1 + list(set(jobs2) - set(jobs1))

In [ ]:
def toMapJob(job):
  return jobs.index(job)

In [ ]:
train_dataset['first_job'] = train_dataset.first_job.apply(toMapJob)
train_dataset['second_job'] = train_dataset.second_job.apply(toMapJob)

test_dataset['first_job'] = test_dataset.first_job.apply(toMapJob)
test_dataset['second_job'] = test_dataset.second_job.apply(toMapJob)

In [ ]:
#train_dataset = train_dataset.drop(['car_licence_plate','company','phone_number','name','address','application_date','credit_card_expire','current_job','credit_card_number'],axis=1)

#test_dataset = test_dataset.drop(['car_licence_plate','company','phone_number','name','address','application_date','credit_card_expire','current_job','credit_card_number'],axis=1)
train_dataset = train_dataset.drop(['car_licence_plate','company','phone_number','name','address','application_date','credit_card_expire','current_job','credit_card_number'],axis=1)

test_dataset = test_dataset.drop(['car_licence_plate','company','phone_number','name','address','application_date','credit_card_expire','current_job','credit_card_number'],axis=1)

Review the changes in each variable

In [ ]:
print("Numero Filas y Columnas en dataset de entrenamiento = ",train_dataset.shape)
print("Valores nulos en dataSet de entrenamiento: \n")
print(train_dataset.isnull().sum())
print("Tipos de valores: \n")
print(train_dataset.dtypes)

In [ ]:
print("Numero Filas y Columnas en dataset de Prueba = ",test_dataset.shape)
print("Valores nulos en dataSet de Prueba: \n")
print(test_dataset.isnull().sum())
print("Tipos de valores: \n")
print(test_dataset.dtypes)

Exploratory Analysis

In [ ]:
corr = train_dataset.corr(method="spearman")
plt.figure(figsize=(15,10)), sns.heatmap(corr, annot = True,cmap='BuGn')
plt.title("Correlation matrix",fontdict= {'fontsize':60})
plt.savefig("correlation-matriz-datSet.png")
plt.show()

In [ ]:
rs = np.random.RandomState(0)
df = pd.DataFrame(rs.rand(20, 20))
corr = train_dataset.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
pandas_profiling.ProfileReport(train_dataset)

In [ ]:
train_dataset = train_dataset.drop(['first_job','second_job','number_of_credit_cards','is_married','smoker','email_domain','credit_card_provider','expire_month','email_domain'],axis=1)

test_dataset = test_dataset.drop(['first_job','second_job','number_of_credit_cards','is_married','smoker','email_domain','credit_card_provider','expire_month','email_domain'],axis=1)

In [ ]:
corr = train_dataset.corr(method="spearman")
plt.figure(figsize=(15,10)), sns.heatmap(corr, annot = True,cmap='BuGn')
plt.title("Correlation matrix",fontdict= {'fontsize':60})
plt.savefig("correlation-matriz-datSet.png")
plt.show()

In [ ]:
train_dataset['defaulted_loan'] = train_dataset['defaulted_loan'].apply(toMapClass)
X_original_train = train_dataset.drop(['application_id','defaulted_loan'], axis=1)
y_original_train = train_dataset['defaulted_loan']
id_original_train = train_dataset['application_id']

X_original_test = test_dataset.drop(['application_id'], axis=1)
id_original_test = test_dataset['application_id']

Evaluate several models with Cross Validation

In [ ]:
# Create Decision Tree Model
desicion_tree = DecisionTreeClassifier()
skf = StratifiedKFold(n_splits=5)
scores_desicionTree = cross_val_score(desicion_tree, X_original_train, y_original_train, cv = skf, scoring="accuracy")

In [ ]:
print("Decision Tree: Accuracies with Cross Validation: ", scores_desicionTree)
print("Decision Tree: Accuracy with Cross Validation: ", scores_desicionTree.mean())

In [ ]:
# Create Suport Vector Machine Model
svc = SVC(kernel='rbf')
skf = StratifiedKFold(n_splits=5)
scores_svc = cross_val_score(svc, X_original_train, y_original_train, cv = skf, scoring="accuracy")

In [ ]:
print("SVC: Accuracies with Cross Validation: ", scores_svc)
print("SVC: Accuracy with Cross Validation: ", scores_svc.mean())

In [ ]:
# Create Naive Bayes Model
naive_Bayes = GaussianNB()
skf = StratifiedKFold(n_splits=5)
scores_naive = cross_val_score(naive_Bayes, X_original_train, y_original_train, cv = skf, scoring="accuracy")
y_pred_cross = cross_val_predict(naive_Bayes,X_original_train,y_original_train,cv = skf )

In [ ]:
print("Naive Bayes: Accuracies with Cross Validation: ", scores_naive)
print("Naive Bayes: Accuracy with Cross Validation: ", scores_naive.mean())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_original_train, y_original_train, test_size=0.3, random_state=1)
def run_model(X_train, X_test, y_train, y_test):
  skf = StratifiedKFold(n_splits=5)
  naive_Bayes = GaussianNB()
  naive_Bayes.fit(X_train, y_train)
  return naive_Bayes

def mostrar_resultados(y_test, pred_y):
  conf_matrix = confusion_matrix(y_test, pred_y)
  plt.figure(figsize=(12, 12))
  sns.heatmap(conf_matrix, annot=True, fmt="d");
  plt.title("Confusion matrix")
  plt.ylabel('True class')
  plt.xlabel('Predicted class')
  plt.show()
  print(classification_report(y_test, pred_y))

In [ ]:
mostrar_resultados(y_original_train,y_pred_cross)

Balance

In [ ]:
os_us = SMOTETomek(ratio=0.5)
X_train_res, y_train_res = os_us.fit_sample(X_train, y_train)
model = run_model(X_train_res, X_test, y_train_res, y_test)
pred_y = model.predict(X_test)
mostrar_resultados(y_test, pred_y)

In [ ]:
y_pred = model.predict_proba(X_train)
skplt.metrics.plot_lift_curve(y_train, y_pred, title='Lift Curve')
plt.show()

In [ ]:
y_pred = model.predict_proba(X_train)
skplt.metrics.plot_roc(y_train, y_pred,title='ROC Curve')
plt.show()

Predict new data

In [ ]:
y_predicted_test = model.predict(X_original_test)
id_original_test = list(id_original_test)
output = pd.DataFrame({'application_id': id_original_test, 'defaulted_loan':y_predicted_test})

Export dataset

In [ ]:
output.to_csv('Hernan Cadena 02-09-2020 Proy final machine learning.csv', index=False) 
#files.download('Hernan Cadena 02-09-2020 Proy final machine learning.csv')